In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math

from pprint import pprint

from pricing.models.abtests import *
from pricing.utils import convert_numeric
# from src.utils import *
from pprint import pprint
import os
import plotly.express as px

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pricing.operations.experiments import level_1_kind_1_courses_1

print("Using as base_host: ", os.environ["PRICING_API_HOST"])

pd.set_option('display.max_columns', None)
from datetime import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using as base_host:  http://10.2.142.113:3000/


In [3]:
df_all = load_raw_test_data(spark,from_cache = True)
print(df_all.shape[0])
df_all.head()

13/02/2020 15:06 - DEBUG - No specific list of alternatives ... downloading test data from all alternatives of experiments: [56, 57, 58, 59, 60, 61, 62, 63, 65]


Folder path:  /home/pricing/cache/
Cache mode!!
Loading cached file for filename: alternatives_data
211121


,order_id,offer_id,registered_at,fee_experiment_id,alternative_id,alternative_ratio,alternative,customer_id,checkout_step,price,value,original_value,discount_value,created_at,university_id,offered_price,exchange,coupon_id,origin,city_id,city,baseline_value,created_day
0,9275216,42268373,2020-01-11 00:20:46.868062,57,324,100.0,baseline,406841705,registered,183.46,183.46,366.91,NaN,2020-01-11 00:19:50.937233,25,366.97,False,NaN,Quero Bolsa,10871.0,Santana de Parnaíba,183.46,2020-01-11
1,9275580,40271054,2020-01-11 01:01:45.473867,58,329,100.0,baseline,405093800,registered,346.15,346.15,346.15,NaN,2020-01-11 00:58:09.443061,61,113.23,False,NaN,Quero Bolsa,6364.0,Brasília,346.15,2020-01-11
2,9275720,42114746,2020-01-11 01:22:29.578114,59,334,100.0,baseline,406836969,registered,490.00,490.00,490.00,NaN,2020-01-11 01:16:27.543660,288,245.00,False,NaN,Quero Bolsa,10901.0,São Paulo,490.00,2020-01-11
3,9276776,42077853,2020-01-11 04:25:20.420630,59,334,100.0,baseline,381999369,registered,348.83,348.83,348.83,NaN,2020-01-11 04:24:52.182255,34,249.17,False,NaN,Quero Bolsa,10901.0,São Paulo,348.83,2020-01-11
4,9277207,40169961,2020-01-11 10:44:23.350075,65,364,100.0,baseline,305285297,registered,245.18,245.18,299.00,NaN,2020-01-11 10:24:38.107967,2738,134.55,False,NaN,Quero Bolsa,NaN,,245.18,2020-01-11


In [5]:
experiment_id = 61
df_exp = df_all[
    (df_all['fee_experiment_id']==experiment_id)&\
    (df_all['alternative']!='dummy')
]
df_exp.sort_values('created_day')


,order_id,offer_id,registered_at,fee_experiment_id,alternative_id,alternative_ratio,alternative,customer_id,checkout_step,price,value,original_value,discount_value,created_at,university_id,offered_price,exchange,coupon_id,origin,city_id,city,baseline_value,created_day
101309,9281067,42168862,2020-01-11 19:36:15.819886,61,344,100.0,baseline,111574917,paid,408.00,408.00,408.00,NaN,2020-01-11 19:31:42.327583,1493,299.20,False,2252644.0,Quero Bolsa,6537.0,Goiânia,408.00,2020-01-11
95075,9281579,39976290,2020-01-11 20:48:55.152380,61,344,100.0,baseline,406686175,paid,296.40,296.40,296.40,NaN,2020-01-11 20:48:18.318549,19,342.00,False,2252520.0,Quero Bolsa,9214.0,Rio de Janeiro,296.40,2020-01-11
96093,9280411,38931221,2020-01-11 18:03:30.819006,61,344,100.0,baseline,400951509,paid,551.09,551.09,551.09,NaN,2020-01-11 17:57:03.230366,1919,258.00,False,2252344.0,Quero Bolsa,6971.0,Belo Horizonte,551.09,2020-01-11
97160,9277224,42164359,2020-01-11 10:36:51.781651,61,344,100.0,baseline,407227494,registered,261.00,261.00,261.00,NaN,2020-01-11 10:34:59.519017,599,315.00,False,NaN,Quero Bolsa,6238.0,Fortaleza,261.00,2020-01-11
145721,9282086,34432528,2020-01-11 22:18:02.883474,61,344,100.0,baseline,405815009,paid,336.00,336.00,336.00,NaN,2020-01-11 22:09:42.113082,3272,624.00,False,2252628.0,Quero Bolsa,10445.0,Campinas,336.00,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201669,9786571,54207843,2020-02-07 13:02:22.288819,61,344,80.0,baseline,412667759,commited,161.50,161.50,161.50,NaN,2020-02-07 12:53:47.221010,75,186.35,False,2347313.0,Quero Bolsa,6098.0,Salvador,161.50,2020-02-07
4253,9785743,53871755,2020-02-07 12:00:20.505045,61,344,80.0,baseline,418395678,paid,336.00,336.00,336.00,NaN,2020-02-07 11:57:13.852478,3272,490.75,False,2350680.0,Quero Bolsa,6971.0,Belo Horizonte,336.00,2020-02-07
163643,9782064,43009632,2020-02-07 00:59:14.232466,61,344,80.0,baseline,407939107,registered,218.40,218.40,218.40,NaN,2020-02-07 00:56:52.628216,501,252.00,False,NaN,Quero Bolsa,9326.0,Natal,218.40,2020-02-07
197453,9783825,37127889,2020-02-07 03:25:35.484852,61,344,80.0,baseline,201270167,registered,301.00,301.00,301.00,NaN,2020-02-07 03:24:23.871112,821,315.00,False,NaN,Quero Bolsa,5722.0,Manaus,301.00,2020-02-07


In [8]:
df_exp.head()

,order_id,offer_id,registered_at,fee_experiment_id,alternative_id,alternative_ratio,alternative,customer_id,checkout_step,price,value,original_value,discount_value,created_at,university_id,offered_price,exchange,coupon_id,origin,city_id,city,baseline_value,created_day
42,9297131,38931753,2020-01-13 16:35:26.620061,61,344,100.0,baseline,407785848,paid,426.13,426.13,426.13,NaN,2020-01-13 16:32:03.190401,1919,199.50,False,2255296.0,Quero Bolsa,6098.0,Salvador,426.13,2020-01-13
57,9303584,37323520,2020-01-13 22:06:10.442156,61,344,100.0,baseline,848604,paid,225.70,225.70,225.70,NaN,2020-01-13 21:59:50.638023,3014,213.50,False,2256457.0,Quero Bolsa,10901.0,São Paulo,225.70,2020-01-13
60,9305817,42288761,2020-01-14 00:35:46.604976,61,344,100.0,baseline,28073,registered,316.66,316.66,316.66,NaN,2020-01-14 00:02:03.619269,29,289.90,False,NaN,Quero Bolsa,10901.0,São Paulo,316.66,2020-01-14
73,9311449,42295204,2020-01-14 13:21:45.147772,61,344,100.0,baseline,407704953,paid,470.58,470.58,470.58,NaN,2020-01-14 13:20:00.881449,36,75.90,False,2257956.0,Quero Bolsa,10726.0,Osasco,470.58,2020-01-14
93,9319597,42166048,2020-01-14 19:40:47.253627,61,344,100.0,baseline,408236578,registered,326.53,326.53,326.53,NaN,2020-01-14 19:39:45.711126,34,466.47,False,NaN,Quero Bolsa,10901.0,São Paulo,326.53,2020-01-14


In [22]:
aux = df_exp[['registered_at','alternative','customer_id']]
aux['registered_day'] = pd.to_datetime(aux['registered_at'].dt.date)
aux.set_index('registered_day', inplace=True)
aux

,registered_at,alternative,customer_id
registered_day,,,
2020-01-13,2020-01-13 16:35:26.620061,baseline,407785848
2020-01-13,2020-01-13 22:06:10.442156,baseline,848604
2020-01-14,2020-01-14 00:35:46.604976,baseline,28073
2020-01-14,2020-01-14 13:21:45.147772,baseline,407704953
2020-01-14,2020-01-14 19:40:47.253627,baseline,408236578
...,...,...,...
2020-02-04,2020-02-04 16:37:10.006760,baseline,417356302
2020-02-04,2020-02-04 20:14:39.074011,baseline,370040997
2020-02-04,2020-02-04 22:27:39.997394,baseline,417512827


In [38]:
df = df_exp.copy()
df['registered_day'] = pd.to_datetime(df['registered_at'].dt.date )
# df['paid_day'] = pd.to_datetime(df['paid_at'].dt.date )
df_by_registered = df.sort_values('registered_at')
# df_by_registered['registered_day_index'] = df_by_registered['registered_day']
# df_by_registered.set_index('registered_day_index', inplace=True)
df_by_registered.set_index('registered_day', inplace=True)

column_data = df_by_registered.groupby('alternative')['customer_id'].rolling('7d').apply(lambda x: x.nunique())

# column_data
df_by_registered

,order_id,offer_id,registered_at,fee_experiment_id,alternative_id,alternative_ratio,alternative,customer_id,checkout_step,price,value,original_value,discount_value,created_at,university_id,offered_price,exchange,coupon_id,origin,city_id,city,baseline_value,created_day
registered_day,,,,,,,,,,,,,,,,,,,,,,,
2020-01-11,9275014,42028031,2020-01-11 00:01:55.029404,61,344,100.0,baseline,116362093,paid,151.22,151.22,151.22,NaN,2020-01-11 00:00:06.141025,24,209.15,False,2251345.0,Quero Bolsa,10901.0,São Paulo,151.22,2020-01-11
2020-01-11,9275015,37128937,2020-01-11 00:03:23.631193,61,344,100.0,baseline,407130061,registered,301.00,301.00,301.00,NaN,2020-01-11 00:00:17.292042,821,315.00,False,NaN,Quero Bolsa,5722.0,Manaus,301.00,2020-01-11
2020-01-11,9275125,37128645,2020-01-11 00:12:08.240248,61,344,100.0,baseline,86358972,registered,206.40,206.40,206.40,NaN,2020-01-11 00:10:43.889126,821,216.00,False,NaN,Quero Bolsa,5722.0,Manaus,206.40,2020-01-11
2020-01-11,9275184,39911326,2020-01-11 00:16:50.669080,61,344,100.0,baseline,406810184,registered,411.48,411.48,411.48,NaN,2020-01-11 00:16:27.409400,25,320.04,False,NaN,Quero Bolsa,10901.0,São Paulo,411.48,2020-01-11
2020-01-11,9275174,40787350,2020-01-11 00:17:35.554530,61,344,100.0,baseline,407138486,registered,74.85,74.85,74.85,NaN,2020-01-11 00:15:35.917008,2378,424.15,False,NaN,Quero Bolsa,10901.0,São Paulo,74.85,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-07,9788729,54739062,2020-02-07 14:58:23.680337,61,346,10.0,seasonality plus,408096762,paid,543.68,543.68,543.68,NaN,2020-02-07 14:53:32.709550,1326,560.15,False,2347833.0,Quero Bolsa,6971.0,Belo Horizonte,494.25,2020-02-07
2020-02-07,9788835,38930469,2020-02-07 14:59:57.548330,61,344,80.0,baseline,376156425,registered,486.54,486.54,486.54,NaN,2020-02-07 14:58:59.419462,1919,344.50,False,NaN,Quero Bolsa,6971.0,Belo Horizonte,486.54,2020-02-07
2020-02-07,9788919,37428051,2020-02-07 15:05:10.336958,61,344,80.0,baseline,399412203,commited,275.97,275.97,275.97,NaN,2020-02-07 15:04:28.476544,2259,356.00,False,2347865.0,Quero Bolsa,10901.0,São Paulo,275.97,2020-02-07


In [9]:
df = df_exp.copy()
df['registered_day'] = pd.to_datetime(df['registered_at'].dt.date )
# df['paid_day'] = pd.to_datetime(df['paid_at'].dt.date )
df_by_registered = df.sort_values('registered_at')
df_by_registered['registered_day_index'] = df_by_registered['registered_day']
df_by_registered['cumsum_new_customer_id'] = df_by_registered.groupby('alternative')['customer_id'].transform(lambda x: ((~x.duplicated())&(pd.isna(x)==False)).cumsum())
df_by_registered['cumsum_price'] = df_by_registered.groupby(['alternative'])['price'].transform(lambda x: x.cumsum())
df_by_registered['cumsum_offered_price'] = df_by_registered.groupby(['alternative'])['offered_price'].transform(lambda x: x.cumsum())
df_by_registered.set_index('registered_day_index', inplace=True)
# df_by_registered['cumsum_new_customer_id_7'] = df_by_registered.groupby('alternative')['customer_id'].rolling('7d').apply(lambda x: x.nunique())
column_data = df_by_registered.groupby('alternative')['customer_id'].rolling('7d').apply(lambda x: x.nunique())
# print(column_data)
df_by_registered.set_index('alternative', inplace=True)
df_by_registered['cumsum_new_customer_id_7'] = column_data
# processed_df = None
# for alternative in df_by_registered.alternative.unique():
#     alternative_df = df_by_registered[df_by_registered['alternative'] == alternative]
#    alternative_df['rollingsum_7_new_customer_id'] = alternative_df['customer_id'].rolling('7d').apply(lambda x: len(np.unique(x)))
# #    alternative_df['rollingsum_14_new_customer_id'] = alternative_df['customer_id'].rolling('14d').apply(lambda x: len(np.unique(x)))
#     alternative_df['rollingsum_7_new_customer_id'] = alternative_df['customer_id'].rolling('7d').apply(lambda x: x.nunique())

df_by_registered['customer_id']


ValueError: cannot handle a non-unique multi-index!

In [ ]:
df = df_exp.copy()
df['registered_day'] = pd.to_datetime(df['registered_at'].dt.date )
df['paid_day'] = pd.to_datetime(df['paid_at'].dt.date )
df_by_registered = df.sort_values('registered_at')
df_by_registered['registered_day_index'] = df_by_registered['registered_day']
df_by_registered['cumsum_new_customer_id'] = df_by_registered.groupby('alternative')['customer_id'].transform(lambda x: ((~x.duplicated())&(pd.isna(x)==False)).cumsum())
df_by_registered['cumsum_price'] = df_by_registered.groupby(['alternative'])['price'].transform(lambda x: x.cumsum())
df_by_registered['cumsum_offered_price'] = df_by_registered.groupby(['alternative'])['offered_price'].transform(lambda x: x.cumsum())
df_by_registered.set_index('registered_day_index', inplace=True)
# df_by_registered['cumsum_new_customer_id_7'] = df_by_registered.groupby('alternative')['customer_id'].rolling('7d').apply(lambda x: x.nunique())
column_data = df_by_registered.groupby('alternative')['customer_id'].rolling('7d').apply(lambda x: x.nunique())
print(column_data)
df_by_registered.set_index('alternative', inplace=True)
df_by_registered['cumsum_new_customer_id_7'] = column_data
processed_df = None
for alternative in df_by_registered.alternative.unique():
    alternative_df = df_by_registered[df_by_registered['alternative'] == alternative]
#    alternative_df['rollingsum_7_new_customer_id'] = alternative_df['customer_id'].rolling('7d').apply(lambda x: len(np.unique(x)))
#    alternative_df['rollingsum_14_new_customer_id'] = alternative_df['customer_id'].rolling('14d').apply(lambda x: len(np.unique(x)))
    alternative_df['rollingsum_7_new_customer_id'] = alternative_df['customer_id'].rolling('7d').apply(lambda x: x.nunique())
    alternative_df['rollingsum_14_new_customer_id'] = alternative_df['customer_id'].rolling('14d').apply(lambda x: x.nunique())
    if processed_df is None:
        processed_df = alternative_df
    else:
        processed_df = pd.concat([processed_df, alternative_df])
df_by_registered = processed_df
df_by_registered.reset_index()
df_agg_registered =  df_by_registered.groupby(['alternative','registered_day']).agg({
'cumsum_new_customer_id': 'last',
'cumsum_offered_price':'last',
'cumsum_price':'last',
'order_id':'count', #counting number of orders per day
'alternative_ratio':'max',
'rollingsum_7_new_customer_id': 'last',
'rollingsum_14_new_customer_id': 'last'
})
